<a href="https://colab.research.google.com/github/ray-islam/generativeAI/blob/main/Simple_LLM_Report_Summarization-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Simple LLM Report Summarization

# Hard reset the broken wheels (DO THIS FIRST)
!pip -q uninstall -y numpy
!pip -q uninstall -y transformers tokenizers safetensors accelerate
!pip -q uninstall -y scipy scikit-learn pandas
!pip -q cache purge

# Install a compatible stack for Colab
!pip -q install --no-cache-dir "numpy==2.1.3"
!pip -q install --no-cache-dir "transformers==4.44.2" "tokenizers==0.19.1" "accelerate==0.33.0" "safetensors>=0.4.3" "pypdf==4.2.0"

# Restart runtime (MANDATORY)
import os, sys
os.kill(os.getpid(), 9)

# Sanity check (must pass before any summarization code)
import numpy as np
import transformers
import tokenizers
import accelerate

print("numpy:", np.__version__)
print("transformers:", transformers.__version__)
print("tokenizers:", tokenizers.__version__)
print("accelerate:", accelerate.__version__)

#Imports + File picker from /content
import re, time, textwrap
from datetime import datetime
from pathlib import Path

import torch
from pypdf import PdfReader
from transformers import pipeline, AutoTokenizer

def clean_text(s: str) -> str:
    s = s.replace("\x00", " ")
    s = re.sub(r"\s+", " ", s).strip()
    return s

def read_pdf_text(pdf_path: str) -> str:
    reader = PdfReader(pdf_path)
    pages = []
    for p in reader.pages:
        pages.append(p.extract_text() or "")
    return "\n".join(pages)

def read_txt_text(txt_path: str) -> str:
    return Path(txt_path).read_text(encoding="utf-8", errors="ignore")

def find_files(base_dir="/content", exts=(".pdf", ".txt")):
    base = Path(base_dir)
    found = []
    for ext in exts:
        found.extend(base.rglob(f"*{ext}"))
    return sorted(found)

def choose_file_interactive(base_dir="/content"):
    files = find_files(base_dir=base_dir, exts=(".pdf", ".txt"))
    if not files:
        raise FileNotFoundError(f"No .pdf/.txt found under {base_dir}. Upload in the Files panel.")
    print("Found files:\n")
    for i, p in enumerate(files, 1):
        print(f"{i:>2}. {p}")
    idx = int(input("\nEnter file number to summarize: ").strip())
    if idx < 1 or idx > len(files):
        raise ValueError("Invalid selection.")
    return str(files[idx-1])

# Build summarizer
MODEL_ID = "facebook/bart-large-cnn"

device = 0 if torch.cuda.is_available() else -1
print("CUDA available:", torch.cuda.is_available())

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

summarizer = pipeline(
    "summarization",
    model=MODEL_ID,
    tokenizer=tokenizer,
    device=device
)

# Token-aware chunking + summarization
def chunk_by_tokens(text: str, tokenizer, max_input_tokens: int = 900, overlap_tokens: int = 60):
    text = clean_text(text)
    if not text:
        return []

    ids = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0
    while start < len(ids):
        end = min(start + max_input_tokens, len(ids))
        chunk_ids = ids[start:end]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        chunks.append(chunk_text.strip())

        if end == len(ids):
            break
        start = max(0, end - overlap_tokens)

    return chunks

def summarize_chunks(summarizer, chunks, max_len=160, min_len=60, sleep_s=0.0):
    out = []
    n = len(chunks)
    for i, c in enumerate(chunks, 1):
        if len(c) < 200:
            continue
        res = summarizer(
            c,
            max_length=max_len,
            min_length=min_len,
            do_sample=False,
            truncation=True
        )[0]["summary_text"]
        out.append(res.strip())
        print(f"✓ Chunk {i}/{n} summarized.")
        if sleep_s:
            time.sleep(sleep_s)
    return out

def final_compress(summarizer, text: str, tokenizer, max_input_tokens=700):
    text = clean_text(text)
    if not text:
        return ""
    chunks = chunk_by_tokens(text, tokenizer, max_input_tokens=max_input_tokens, overlap_tokens=40)
    finals = summarize_chunks(summarizer, chunks, max_len=200, min_len=80)
    return clean_text(" ".join(finals))

# Run end-to-end + save/download

from google.colab import files

path = choose_file_interactive("/content")
print("\nSelected:", path)

if path.lower().endswith(".pdf"):
    raw_text = read_pdf_text(path)
elif path.lower().endswith(".txt"):
    raw_text = read_txt_text(path)
else:
    raise ValueError("Pick a .pdf or .txt")

raw_text = clean_text(raw_text)
print(f"\nLoaded {len(raw_text):,} characters.")
print(raw_text[:800] + ("..." if len(raw_text) > 800 else ""))

CHUNK_INPUT_TOKENS = 900
chunks = chunk_by_tokens(raw_text, tokenizer, max_input_tokens=CHUNK_INPUT_TOKENS, overlap_tokens=60)
print("\nTotal chunks:", len(chunks))

chunk_summaries = summarize_chunks(summarizer, chunks, max_len=160, min_len=60)

intermediate = "\n\n".join(chunk_summaries)
print("\n--- Intermediate (first 800 chars) ---\n")
print(intermediate[:800] + ("..." if len(intermediate) > 800 else ""))

final_summary = final_compress(summarizer, intermediate, tokenizer, max_input_tokens=700)

print("\n======= EXECUTIVE SUMMARY =======\n")
print(textwrap.fill(final_summary, 100))

prompt = (
    "HEADLINE AND BULLETS:\n"
    + final_summary
    + "\n\nWrite a short headline and 5 bullet points with key findings."
)
headline_bullets = summarizer(prompt, max_length=220, min_length=120, do_sample=False, truncation=True)[0]["summary_text"]

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
base = Path(path).stem

out_exec = f"{base}_executive_summary_{stamp}.txt"
out_chunks = f"{base}_chunk_summaries_{stamp}.txt"
out_hb = f"{base}_headline_bullets_{stamp}.txt"

Path(out_exec).write_text(final_summary, encoding="utf-8")
Path(out_chunks).write_text(intermediate, encoding="utf-8")
Path(out_hb).write_text(headline_bullets, encoding="utf-8")

files.download(out_exec)
files.download(out_chunks)
files.download(out_hb)

